In [1]:
%env ALL_PROXY=http://127.0.0.1:7890
%env HTTP_PROXY=http://127.0.0.1:7890
%env HTTPS_PROXY=http://127.0.0.1:7890

env: ALL_PROXY=http://127.0.0.1:7890
env: HTTP_PROXY=http://127.0.0.1:7890
env: HTTPS_PROXY=http://127.0.0.1:7890


In [2]:
%env HF_HUB_CACHE=./data/hf_cache

env: HF_HUB_CACHE=./data/hf_cache


# Text classification example

## Step 01. import related packages

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/ubuntu/MyFiles/GitHub/demos/transformers/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 02. Load data

In [4]:
import os
import pandas as pd

data = pd.read_csv(os.path.join('.', 'data', 'train', 'ChnSentiCorp_htl_all.csv'))
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [5]:
data.shape

(7766, 2)

In [6]:
data = data.dropna()

In [7]:
data.shape

(7765, 2)

## Step 03. Create dataset

In [10]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv(os.path.join('.', 'data', 'train', 'ChnSentiCorp_htl_all.csv'))
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]['review'], int(self.data.iloc[index]['label'])

    def __len__(self):
        return len(self.data)

In [11]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step 04. Split the dataset

In [12]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.8, 0.2])
len(trainset), len(validset)

(6212, 1553)

In [14]:
for i in range(5):
    print(trainset[i])

('该酒店是我住过的最差的酒店，房间设施陈旧，一股霉味，床单被褥粘糊糊的，最可气的是服务员绝然会栽赃陷害住客，我在那一共住了4天，7月20日结帐的时候前台说客房服务员检查发现有个付费毛巾被开封，我经常出差，住过无数酒店，还第一次遇到这种事情，我住酒店从不动那些付费用品，房间就我一个人住，其间没有任何访客，谁会去打开那个破包装毛巾，开了一个小口又不用，可酒店服务员硬说是我打开的，一个酒店硬件设施差点没什么，服务质量差点也有情可原，但服务员不诚实，甚至栽赃顾客是有本质问题的，奉劝大家千万不要住这家酒店。', 0)
('还没有说完，因为这次是一个人住，也不想去外面吃，就来到她的中餐厅叫怡和轩还是什么的我忘记了，主攻粤菜。也是给我感动的一塌糊涂！一个人能吃到什么？一个女孩子更加难点菜。她们的服务员把我给照顾的无微不至！细节就不说了，最后吃完还送我下楼到门口，其实我是房客，认得路的呀！呵呵！美眉漂亮不说，很温柔体贴，这都是上层的管理和制度的导向，我认为海悦确实是以人为本，全心全意的为客户着想，比起那些所谓的5星级不知道好多少倍！前两天在锦江住店我还生了一回气呢！物超所值！我下次还来海悦啊！谢谢！补充点评2007年9月13日：地方偏不是问题，她的门童会很负责的帮你叫车。反正我每次出去他都帮我准备好了。我喜欢这里的幽静！', 1)
('周围的环境很好.在房间可以看到大海.普通间很一般.建议住豪华间.价格差不了多少.豪华间非常大.放三张大床也有够大的空间.非常不错.还可以看到大海', 1)
('房间还是对得起这个价格，优点是这里的每间房间都是正对三亚湾的海。缺点就是酒店没泳池，住一晚还行，但作为度假，酒店设施硬件差了点，其实酒店就是一整幢大楼，没绿化没游泳池那种。还有个优点，酒店能免费接机，门口公交很多，去大东海，去亚龙湾都有公交车，这点很不错，节省了很多打车费用。补充点评2008年6月30日：对了，这个酒店的排水系统真的很差，马桶都冲不干净，让人心情因为马桶而变差。。。。希望改进。', 1)
('出于不信邪的原因，入住了该酒店。之前查阅了携程上的点评，最近的点评已经有大半年时间了，一厢情愿的以为酒店设施有所改善。但是，去了只能用三个来形容，差！差！差！一次恐怖的入宿。电梯依旧吓人，房间一股霉味，一个灯是坏的！第二天就换到隔壁的一百假日去了。', 0)


## Step 05. Create dataloader

In [30]:
import torch

tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')

def collate_fn(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    inputs['labels'] = torch.tensor(labels)
    return inputs

In [31]:
from torch.utils.data import DataLoader

train_loader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(validset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [32]:
next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101, 1765, 4157,  ..., 1372, 2372,  102],
        [ 101, 5018,  671,  ..., 6132, 3302,  102],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        ...,
        [ 101, 2600,  860,  ...,  702, 3302,  102],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101,  100, 2345,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 0])}

## Step 06. Create the model and optimizer

In [33]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained('hfl/rbt3')

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step 07. Train and validate

In [35]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch['labels'].long()).float().sum()
    return acc_num / len(validset)


def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step == 0 or global_step % log_step == 0:
                print(f"ep: {ep}, Step {global_step}: loss = {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, Validation Accuracy = {acc}")

## Step 08. Train the model

In [36]:
train(epoch=10, log_step=100)

ep: 0, Step 0: loss = 0.7489166855812073
ep: 0, Step 100: loss = 0.3359348177909851
ep: 0, Validation Accuracy = 0.8795879483222961
ep: 1, Step 200: loss = 0.33741503953933716
ep: 1, Step 300: loss = 0.2895704507827759
ep: 1, Validation Accuracy = 0.8963297009468079
ep: 2, Step 400: loss = 0.17358046770095825
ep: 2, Step 500: loss = 0.2642514705657959
ep: 2, Validation Accuracy = 0.8969736099243164
ep: 3, Step 600: loss = 0.41686078906059265
ep: 3, Step 700: loss = 0.2410842776298523
ep: 3, Validation Accuracy = 0.8873149156570435
ep: 4, Step 800: loss = 0.06928562372922897
ep: 4, Step 900: loss = 0.16453737020492554
ep: 4, Validation Accuracy = 0.8905344605445862
ep: 5, Step 1000: loss = 0.02317120134830475
ep: 5, Step 1100: loss = 0.11493042856454849
ep: 5, Validation Accuracy = 0.8873149156570435
ep: 6, Step 1200: loss = 0.01408088207244873
ep: 6, Step 1300: loss = 0.05626334622502327
ep: 6, Validation Accuracy = 0.891822338104248
ep: 7, Step 1400: loss = 0.14026986062526703
ep: 7, 

## Step 09. Model prediction

In [43]:
sentence = "I Think this hotel is good!"
# sentence = "I think this hotel is terrible..."
id2label = {0: 'bad', 1: 'good'}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sentence, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"Input: {sentence}\nModel Prediction: {id2label.get(pred.item())}")

Input: I Think this hotel is good!
Model Prediction: good


In [46]:
from transformers import pipeline

model.config.id2label = id2label
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [47]:
pipe(sentence)

[{'label': 'good', 'score': 0.7181729674339294}]